# 18-819 Project, Fall 2022

Team Members:

- Claudio Gomes ()
- Ke Xu ()
- Yuxuan Zheng (yzheng5@andrew.cmu.edu)

## Introduction

## Problem Formulation

## Classical Approach

In [4]:
# Set up the environment
%pip install cvxpy
%pip install cvxopt
%pip install dwave-system
## Note: for dwave, you need to run dwave setup and put your API key.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


You are able to run the classical solver on CPU with the following command:

In [1]:
## Uncomment if you want to use CPLEX and generate .lp file of the model.
## By default, it uses CVXOPT.
## Args '-C' or '--CPLEX' make the function use CPLEX and generate .lp file.
# !python classical/solver_mip.py -v -C
!python classical/solver_mip.py -v

2022-11-25 20:12:21 | main | DEBUG: Received input argument: Namespace(verbose=True, CPLEX=True)
2022-11-25 20:12:21 | main | DEBUG: Project path: C:\Users\Work\Documents\GitHub\18819-project
2022-11-25 20:12:21 | main | INFO: Num of weights: 3
2022-11-25 20:12:21 | main | INFO: Num of biases: 3
2022-11-25 20:12:21 | main | INFO: Considering: name: linear_relu_stack.0, layer: Linear(in_features=784, out_features=20, bias=True)
2022-11-25 20:12:21 | main | INFO:   lower bound: 
[[-0.4234205800365508 -1.8351959504381299 0.8804067259743539
  -0.8196989746911543 -3.793728826809773 -1.8864171617425003
  -0.5739344345791544 0.8865537540367369 -1.6154333620130206
  -0.08706178086480955 -2.3534664022913234 -2.0912231290293244
  -1.7001457389483525 -2.3174443445543202 -0.24076084230795802
  0.11701600115205879 -2.1210138638766782 -1.9108620069399005
  -3.217863043469488 10.206187134601869]]
2022-11-25 20:12:21 | main | INFO:   upper bound: 
[[4.702268312221214 0.952284438434611 6.08921643174151

## Quantum Approach

In [6]:
## Uncomment if you want to calculate a new sampleset.
## By default, it uses last obtained sampleset.
## Args '-s' or '--sampleset' make the function calculate new sampleset.
# !python quantum/solver_dwave.py -v -s
!python quantum/solver_dwave.py -v

2022-11-25 20:11:46 | main | DEBUG: Received input argument: Namespace(verbose=True, sampleset=False)
2022-11-25 20:11:46 | main | DEBUG: Project path: C:\Users\Work\Documents\GitHub\18819-project
2022-11-25 20:11:46 | main | INFO: 43 feasible solutions of 62.
2022-11-25 20:11:46 | main | INFO: Best sample: Sample(sample={'x1': -0.0, 'x10': 1.2686546325486001, 'x100': 0.1, 'x101': 0.0, 'x102': 0.0, 'x103': 0.0, 'x104': 0.1, 'x105': 0.0, 'x106': 0.0, 'x107': 0.09999999999999999, 'x108': 0.0, 'x109': 0.0, 'x11': 1.8706970922190018, 'x110': 0.0, 'x111': 0.1, 'x112': 0.0, 'x113': 0.1, 'x114': 0.1, 'x115': 0.1, 'x116': 0.1, 'x117': 0.1, 'x118': 0.09999999999999999, 'x119': 0.09999999999999999, 'x12': 2.7112278645721823, 'x120': 0.1, 'x121': 0.1, 'x122': 0.1, 'x123': 0.1, 'x124': 0.1, 'x125': 0.0, 'x126': 0.0, 'x127': 0.0, 'x128': 0.0, 'x129': 0.0, 'x13': -0.0, 'x130': 0.0, 'x131': 0.0, 'x132': 0.0, 'x133': 0.0, 'x134': 0.0, 'x135': 0.0, 'x136': 0.1, 'x137': 0.0, 'x138': 0.0, 'x139': 0.1, 'x

## Comparison